# Observability and reachability matrix

In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage
from tvsclib.system_identification_svd import SystemIdentificationSVD
from tvsclib.toeplitz_operator import ToeplitzOperator
from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg

import tvsclib.utils as utils

The reachability matrix $\mathcal{R}_k$ and observability matrix $\mathcal{O}_k$ represent the mapping from the inputs to the state $x_k$ and from the state $x_k$ to the output. 
These matrices are related to the Hankel operator accoriding to

$$H_k = \mathcal{O}_k \mathcal{R}_k $$


These are matrices taken from the (strictly) upper or lower triangular blockmatrices as illustrated here:


$$
\begin{bmatrix}
\begin{matrix}
   D_1  \\
\end{matrix}&
\boxed{\begin{matrix}
   C_1B_2 & C_1A_2B_3 & C_1A_2A_3B_4\\
\end{matrix}}
\\
\boxed{\begin{matrix}
   C_2B_1 \\
   C_3A_2B_1 \\
   C_4A_3A_2B_1  
\end{matrix}}&
\begin{matrix}
 D_2 &C_2B_3 & C_2A_3B_4 \\
 C_3B_2&  D_3 & C_3B_4 \\
 C_4A_3B_2 &  C_4B_3 & D_4
\end{matrix}
\end{bmatrix}
\quad\quad
\begin{bmatrix}
\begin{matrix}
   D_1 &  C_1B_2 \\
   C_2B_1  & D_2
\end{matrix}&
\boxed{\begin{matrix}
   C_1A_2B_3 & C_1A_2A_3B_4\\
   C_2B_3 & C_2A_3B_4
\end{matrix}}
\\
\boxed{\begin{matrix}
   C_3A_2B_1 & C_3B_2 \\
   C_4A_3A_2B_1 & C_4A_3B_2
\end{matrix}}&
\begin{matrix}
   D_3 & C_3B_4 \\
   C_4B_3 & D_4
\end{matrix}
\end{bmatrix}
$$

Lets illustrate this here with an rank one matrix. 

Here we will first only consider causal systems

In [ ]:
matrix = np.arange(0,12).reshape((-1,1))@np.arange(0,12).reshape((1,-1))
dims_in =  np.array([2, 1, 2, 1])*2
dims_out = np.array([1, 2, 1, 2])*2
T = ToeplitzOperator(matrix, dims_in, dims_out)
S = SystemIdentificationSVD(T)#,epsilon=1e-6)
system = MixedSystem(S).causal_system
system_anti = MixedSystem(S).anticausal_system


display(matrix)

utils.show_system(system)
plt.clim(0,11**2)

Now we extract the hankel matrix for the timestep $k$ and display it

In [ ]:
k =2
# Number of input dims is the sum of the input dims up to the stage k (excluding k itself)
d_in = np.sum(dims_in[:k])
# Number of output dims is the sum of the output dims of stage k and up
d_out = np.sum(dims_out[k:])

#now extract the according matrix
H_caus = matrix[np.sum(dims_out)-d_out:,:d_in] #indexing with negative indices does lead to problems in certian cases
plt.matshow(H_caus)
plt.clim(0,11**2)
display(H_caus)

Now we can check if $H_k = 𝓞_k 𝓡_k $

In [ ]:
np.max(np.abs(H_caus-(system.observability_matrix(k)@system.reachability_matrix(k))))

The same is possible for the anticausal part

In [ ]:
k =0
# Number of input dims is the sum of the input dims from the stage k (excluding k itself) up
d_in = np.sum(dims_in[k+1:])
# Number of output dims is the sum of the output dims of stage k and down
d_out = np.sum(dims_out[:k+1])

print(d_in)
print(d_out)
#now extract the according matrix
H_anticaus = matrix[:d_out,np.sum(dims_in)-d_in:] 
#plt.matshow(H_2)
#plt.clim(0,11**2)
display(H_anticaus)

In [ ]:
np.max(np.abs(H_anticaus-(system_anti.observability_matrix(k)@system_anti.reachability_matrix(k))))

# Some notes on indexing

Some notes to make the indexing more clear:

As defined above the $H_k$, $𝓞_k$ and  $𝓡_k$ are defined by their relation to the state $x_k$.

It is also important to note the definition for the casual system:

$$x_{k+1} = A_k x_k + B_k u_k $$
$$y_k = C_k x_k + D_k u_k $$


This means that $C_k$ is part of $𝓞_k$ as it relates $x_k$ to the output.
But $B_k$ is not part of $𝓡_k$ as it does not relate to $x_k$. 


For a causal system with $n$ stages we have $n+1$ involved stages. The first stage is the input stage $x_1$ or `x[0]`. This state has usually dim $0$.
Then we have the internal stages.
Finallly we have the output stage $x_{n+1}$ or `x[n]`.

We can define $𝓞_k$ and $𝓡_k$ for all of these stages, but these have a 0 dim for the input and the output stage. 
Therefore only the internal states are usually interesting. These have the indices $2,3,4,\cdots,n$ or `1,2,3, ... ,len(stages)-1`

In [ ]:
iter = range(1,len(system.stages))
for i in iter:
    print("k=",i)
    display(system.reachability_matrix(i).shape)
    display(system.observability_matrix(i).shape)

For the anticausal system the definition is


$$x_{k-1} = A_k x_k + B_k u_k $$
$$y_k = C_k x_k + D_k u_k $$

This leaves us with the input state $x_n$ or `x[n-1]` and the output state $x_{0}$ or `x[-1]`.
Simmilarly to bevore the (usaully) intersting states are the internal states.

These have the indices $n \cdots,2,3,1$ or `len(stages)-2, ... ,2,1,0`



In [ ]:
iter = range(len(system_anti.stages)-2,-1,-1)
for i in iter:
    print("k=",i)
    display(system_anti.reachability_matrix(i).shape)
    display(system_anti.observability_matrix(i).shape)

This also has the consequence that the indexing of the Hankel opertors is not symetric:

In the following matrix the $H_3$ operator is marked:
$$
\begin{bmatrix}
\begin{matrix}
   D_1 &  C_1B_2 \\
   C_2B_1  & D_2
\end{matrix}&
\begin{matrix}
   C_1A_2B_3 & C_1A_2A_3B_4\\
   C_2B_3 & C_2A_3B_4
\end{matrix}
\\
\boxed{\begin{matrix}
   C_3A_2B_1 & C_3B_2 \\
   C_4A_3A_2B_1 & C_4A_3B_2
\end{matrix}}&
\begin{matrix}
   D_3 & C_3B_4 \\
   C_4B_3 & D_4
\end{matrix}
\end{bmatrix}
$$

If we now go to the anticausal part we mark the symmetric part. Due to the inbalance of the indexing this is not the second Hankel operator but $H_2$:
$$
\begin{bmatrix}
\begin{matrix}
   D_1 &  C_1B_2 \\
   C_2B_1  & D_2
\end{matrix}&
\boxed{\begin{matrix}
   C_1A_2B_3 & C_1A_2A_3B_4\\
   C_2B_3 & C_2A_3B_4
\end{matrix}}
\\
\begin{matrix}
   C_3A_2B_1 & C_3B_2 \\
   C_4A_3A_2B_1 & C_4A_3B_2
\end{matrix}&
\begin{matrix}
   D_3 & C_3B_4 \\
   C_4B_3 & D_4
\end{matrix}
\end{bmatrix}
$$